In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import numpy as np
from nn_magnetics.data import simulate_demag
from nn_magnetics.utils.physics import demagnetizing_factor

In [40]:
a = 0.5
b = 0.5
c = 1

chi_x = 0.1
chi_y = 0.2
chi_z = 0.3

In [41]:
data = simulate_demag(a, b, (chi_x, chi_y, chi_z), 30)

Meshing magnet and applying demag effects
Creating measurement grid
Calculating analytical B-field
Calculating demag B-field
Calculating reduced field


In [42]:
data["reduced_polarization"]

array([-1.19052578e-19,  1.33338888e-18,  9.44353978e-01])

In [43]:
factor_z = demagnetizing_factor(a, b, c)
factor_x = demagnetizing_factor(c, a, b)
factor_y = demagnetizing_factor(a, c, b)

In [44]:
np.array([0, 0, 1]) * (1 -   np.array([0.1 * factor_x, 0.2 * factor_y, 0.3 * factor_z]))

array([0.        , 0.        , 0.94050515])